### Behavioral Analysis for eLife Reviews
#### Looking at raw and filtered trajectories, distribution of turns, 

In [2]:
# Imports

import numpy as np
import utility_fx as util
import os
from matplotlib import pyplot as plt
import ratterdam_Defaults as Def
import ratterdam_visBasic as Vis
import ratterdam_RepetitionCoreFx as RepCore
import RateMapClass_William_20190308 as RateMapClass
import williamDefaults as wmDef
import pandas as pd
import ratterdam_CoreDataStructures as Core
import ratterdam_ParseBehavior as Parse
import numpy as np
import pandas as pd
from scipy.stats import sem
import utility_fx as util
import os
import matplotlib.gridspec as gridspec
from matplotlib import pyplot as plt
import ratterdam_Defaults as Def
import ratterdam_visBasic as Vis
import placeFieldBorders
import RateMapClass_William_20190308 as RateMapClass
import williamDefaults as wmDef
from matplotlib.backends.backend_pdf import PdfPages
import more_itertools, itertools
from sklearn.metrics import auc
from scipy.interpolate import splrep, splev, PchipInterpolator as pchip
from scipy.spatial import ConvexHull
import scipy
import ratterdam_DataFiltering as Filt
import alleyTransitions as alleyTrans
import newAlleyBounds as nab
import json
from itertools import groupby
import repeatingPC as repPC
import matplotlib.path as path
import copy 



In [3]:
%qtconsole --style paraiso-dark

In [4]:
rat = "R859"
day = "D1"
clustName = "TT1\\cl-maze1.1"
smoothing = 2
vthresh = Def.velocity_filter_thresh

In [5]:
df = f'E:\Ratterdam\\{rat}\\{rat}_RatterdamOpen_{day}\\'
with open(df+"sessionEpochInfo.txt","r") as f:
    lines = f.readlines()
start, end = float(lines[0].split(',')[0]), float(lines[0].split(',')[1])
pos = util.read_pos(df)
ts = np.asarray(sorted(list(pos.keys())))
posx, posy = Parse.adjustPosCamera(df, pos, ts)
position = np.column_stack((ts, posx, posy))
position = position[(position[:,0]>=start) & (position[:,0]<=end)]

#next two lines gets rid of zeros that occur when you lose tracking
# depending on how the camera image is flipped the zeros will either be
# at 0,0 or 640,480
position = position[np.logical_and(position[:,1]>0, position[:,2]>0)]
position = position[np.logical_and(position[:,1]<640, position[:,2]<480)]

In [6]:
base_position = copy.deepcopy(position) # keep a copy of original

In [7]:
# processing step 1 - velocity filter
sv = Filt.computeSpeed(position, winsz=50)
vf_pos = position[sv > vthresh]
removed_points = position[sv <= vthresh] # keep concatenating at each processing step the points youre removing so we can show them all at the end
position = vf_pos


In [8]:
#processing step 2 - behavior outside alleys or intersections
ratborders = nab.loadAlleyBounds(rat, day)

ontrackPositionIdx = np.empty((0), dtype=int)

for regionName, b in ratborders.alleyInterBounds.items():
    ul, ll, ur, lr = [b[0][0], b[1][1]], \
                    [b[0][0], b[1][0]], \
                    [b[0][1], b[1][1]], \
                    [b[0][1], b[1][0]]

    contour = path.Path([ll, lr, ur, ul])
    posIdx = np.where(contour.contains_points(position[:,1:]))[0]

    ontrackPositionIdx = np.concatenate((ontrackPositionIdx, 
                                        posIdx.astype(int)))





ontrackPosition = position[ontrackPositionIdx]
offtrackPositionIdx = [i for i in range(len(position)) if i not in ontrackPositionIdx]
offtrackPosition = position[offtrackPositionIdx]

In [9]:
pos, turns = alleyTrans.alleyTransitions(base_position, ratborders, graph=False)
turns = pd.DataFrame(turns)
turns.columns = ['Allo-','Ego','Allo+','Ts exit','Ts entry', 'Alley-', 'Inter','Alley+']

turns = pd.DataFrame(data=turns)
turns.dropna(inplace=True) 

In [10]:
rewards = RepCore.readinRewards(rat, day)


In [65]:
# this is for filtering pass through visits as opposed to turnarounds 
filteredOut = np.empty((0,2))
filteredIn = np.empty((0,2))

for i in range(1,turns.shape[0]-1):
    row = turns.iloc[i]
    inter = row['Inter']
    # edit 10/2 removing check that last turn's inter wasnt the same,
    # i.e if alley- had a turnaround. since we are looking at things
    # in terms of alley+, only remove a turn if thats where a turnaround was
    #4/4/22 added checking turns ego not equal 0. this corresponds to lost tracking 
    ts_start, ts_end = float(row['Ts entry']), float(turns.iloc[i+1]['Ts exit'])
    behav = ontrackPosition[(ontrackPosition[:,0]>ts_start)&(ontrackPosition[:,0]<=ts_end)]
    behav = behav[(behav[:,1]>0)&(behav[:,2]>0)]
    behav = behav[:,1:] # remove ts, keep pos only
    isReward = np.where(np.asarray([(ts_start < i < ts_end) for i in rewards])==True)[0]

    if row.Ego not in ['0','3'] and turns.iloc[i+1].Inter != inter and len(isReward)==0:
        filteredIn = np.vstack((filteredIn, behav))
    else:
        filteredOut = np.vstack((filteredOut, behav))



